In [1]:
%%time
import os
import pandas as pd
import numpy as np

Wall time: 7.86 s


In [2]:
%%time

# Get working directory
print(os.getcwd())

# Set working directory

os.chdir('C:\\Users\\ravigv\\Documents\\cosine_sim') 
print(os.getcwd())

C:\Users\ravigv
C:\Users\ravigv\Documents\cosine_sim
Wall time: 0 ns


In [3]:
%%time
df = pd.read_excel('5whyQA1.xlsx')

df.head()

Wall time: 388 ms


,Prob_statement,First_Why
0,A la carte users have reported that they are i...,Why users are seeing a schedule error when try...
1,PGE~ABAP : ABAP System not available,Why ABAP System not available?
2,PGE-CI & PGE-App. inaccessible from SAP-GUI ?,Why were PGE-CI & PGE-App. inaccessible from S...
3,Effort involved in manual validation is consid...,Why is the considerable effort involved in man...
4,User unable to pack coil (HU),Why the pack was open


In [5]:
%%time
# Preprocessing

import re
import gensim 
from gensim.parsing.preprocessing import remove_stopwords

#from nltk.stem.lancaster import LancasterStemmer
#st = LancasterStemmer()

def clean_sentence(sentence, stopwords=False):
    
    sentence = sentence.lower().strip()
    sentence = re.sub(r'[^a-z0-9\s]', '', sentence)
    #sentence = re.sub(r'\s{2,}', ' ', sentence)
     # remove words containing numbers
    sentence = re.sub('\w*\d\w*', '', sentence)
    # remove numbers 
    sentence = re.sub(r'[0-9]+','',sentence)
    
    if stopwords:
         sentence = remove_stopwords(sentence)
        
    return sentence

Wall time: 0 ns


In [6]:
%%time
def get_cleaned_sentences1(df, col, stopwords=False):
    
    cleaned_sentences=[]
    
    for index,row in df.iterrows():
#         print(index,row[col])
        cleaned=clean_sentence(row[col],stopwords);
        cleaned_sentences.append(cleaned);
    return cleaned_sentences;
        

        
# Calling the function get_cleaned_sentences

# Case1: Without stopwords
cleaned_sentences=get_cleaned_sentences1(df, 'Prob_statement', stopwords=True)
print(cleaned_sentences);

print("\n")

# Case2: With stopwords
cleaned_sentences_with_stopwords=get_cleaned_sentences1(df, 'Prob_statement', stopwords=False)
print(cleaned_sentences_with_stopwords);

['la carte users reported intermittently getting recommender error trying offers', 'pgeabap abap available', 'pgeci pgeapp inaccessible sapgui', 'effort involved manual validation considerable', 'user unable pack coil hu', 'users able post goods movement', 'new user id added approver', 'change order merge validation failing holding promotion', 'edi file translation failure', 'planning book unavailable business users', 'opsheet communicating fastpass controllers', 'differences fi spl ledger', 'chemistry characteristics updated kingston batch', 'users able access lilo ui', 'able maintain datadeal grouping ipm', 'mediamarket spain invoic idocs got failed', 'users getting service available error accessing mle', 'business users able create sales order delivery billing documents', 'note implementation', 'production order scheduled', 'scanner gun showing blank screen wrong vendor batch number scanned', 'transactions flowing mis transactions stuck queue', 'settle issue', 'production orders ava

In [9]:
%%time

# tokenize
sentences=cleaned_sentences
#sentences=cleaned_sentences

# Split it by white space 
tokens = [[word for word in document.split() ]
         for document in sentences]

# tokens

Wall time: 0 ns


#### Creating Gensim custom word2vec model

In [11]:
# ''' The parameters:
# min_count = int - Ignores all words with total absolute frequency lower than this - (2, 100)
# window = int - The maximum distance between the current and predicted word within a sentence. 
#          E.g. window words on the left and window words on the left of our target - (2, 10)
# size = int - Dimensionality of the feature vectors. - (50, 300)
# sample = float - The threshold for configuring which higher-frequency words are randomly downsampled.Highly influencial. - (0, 1e-5)    
# alpha = float - The initial learning rate - (0.01, 0.05)
# min_alpha = float - Learning rate will linearly drop to min_alpha as training progresses. To set it: alpha - (min_alpha * epochs) ~ 0.00
# negative = int - If > 0, negative sampling will be used, the int for negative specifies how many "noise words" should be drown. If set to 0, no negative sampling is used. - (5, 20)
# workers = int - Use these many worker threads to train the model (=faster training with multicore machines)'''


In [12]:
%%time
# Step1: set up the parameters with word2vec

import multiprocessing
from gensim.models import Word2Vec


# Count the number of cores in a computer
cores = multiprocessing.cpu_count() 

# Call
w2v_model = Word2Vec(min_count=5,
                     window=2,
                     size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)

Wall time: 0 ns


In [13]:
%%time

# Step2: Building the Vocabulary 

w2v_model.build_vocab(tokens)

Wall time: 72 ms


In [14]:
%%time

# Step3:Training of the model:
# Parameters of the training:

# total_examples = int - Count of sentences;
# epochs = int - Number of iterations (epochs) over the corpus - [10, 20, 30]

w2v_model.train(tokens, total_examples=w2v_model.corpus_count, epochs=30)

Wall time: 399 ms


(10, 8190)

In [21]:
%%time
# save word2vec model
w2v_model.save("word2vec_ps.model")

# Save WV
w2v_model.wv.save_word2vec_format("word2vec_ps_model.txt")


Wall time: 7.95 ms


In [17]:
%%time

# Create BOW
# Create corpora
from gensim import corpora
dictionary = corpora.Dictionary(tokens)
# for key, value in dictionary.items():
#     print(key, ' : ', value)


# Creating BOW

import pprint
bow_corpus = [dictionary.doc2bow(text) for text in tokens]
for sent,embedding in zip(sentences,bow_corpus):
    print(sent)
    print(embedding)

la carte users reported intermittently getting recommender error trying offers
[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1)]
pgeabap abap available
[(10, 1), (11, 1), (12, 1)]
pgeci pgeapp inaccessible sapgui
[(13, 1), (14, 1), (15, 1), (16, 1)]
effort involved manual validation considerable
[(17, 1), (18, 1), (19, 1), (20, 1), (21, 1)]
user unable pack coil hu
[(22, 1), (23, 1), (24, 1), (25, 1), (26, 1)]
users able post goods movement
[(9, 1), (27, 1), (28, 1), (29, 1), (30, 1)]
new user id added approver
[(26, 1), (31, 1), (32, 1), (33, 1), (34, 1)]
change order merge validation failing holding promotion
[(21, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1)]
edi file translation failure
[(41, 1), (42, 1), (43, 1), (44, 1)]
planning book unavailable business users
[(9, 1), (45, 1), (46, 1), (47, 1), (48, 1)]
opsheet communicating fastpass controllers
[(49, 1), (50, 1), (51, 1), (52, 1)]
differences fi spl ledger
[(53, 1), (54, 1), (55, 1), 

In [18]:
%%time
#question_orig="do I need to learn algorithms to be a data scientist ?";
query="A la carte users have reported that they are intermittently getting a Recommender error when trying to get offers"
query=clean_sentence(query,stopwords=True);
query_embedding = dictionary.doc2bow(query.split())


print("\n\n",query,"\n",query_embedding)




 la carte users reported intermittently getting recommender error trying offers 
 [(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1)]
Wall time: 1.01 ms


In [19]:
%%time
import sklearn
from sklearn.metrics.pairwise import cosine_similarity

def get_cosim(query,docs):
      
    count = []
    simscore = []
    result = []

    for index,faq_embedding in enumerate(docs):
            sim=cosine_similarity(faq_embedding,query)[0][0]

            count.append(index)
            simscore.append(sim)
            result.append(sentences[index])



    dfScore = pd.DataFrame(list(zip(count, simscore, result)),
                  columns=['id','simscore', 'result'])


    dfsort = dfScore.sort_values('simscore', ascending = False)


    return dfsort


Wall time: 1.62 s


In [35]:
%%time
# Getting Phrase Embeddings from Word Embeddings

def getWordVec(word,model):
        samp=model['users']
        vec=[0]*len(samp);
        try:
                vec=model[word]
        except:
                vec=[0]*len(samp)
        return (vec)


def getPhraseEmbedding(phrase,embeddingmodel):
                       
        samp=getWordVec('users', embeddingmodel)
        vec=np.array([0]*len(samp));
        den=0;
        for word in phrase.split():
            #print(word)
            den=den+1;
            vec=vec+np.array(getWordVec(word,embeddingmodel))
        #vec=vec/den;
        #return (vec.tolist());
        return vec.reshape(1, -1)


Wall time: 0 ns


In [29]:
%%time 
# Load model
word2vec_ps_model = gensim.models.KeyedVectors.load_word2vec_format("./word2vec_ps_model.txt", binary = False)
print("Loaded word2vec_ps_model")

Loaded word2vec_ps_model
Wall time: 2.96 ms


In [36]:
%%time
#With w2Vec_ps model



sent_embeddings=[]
for sent in cleaned_sentences:
    sent_embeddings.append(getPhraseEmbedding(sent,word2vec_ps_model))

question_embedding=getPhraseEmbedding(query,word2vec_ps_model)

get_cosim(question_embedding,sent_embeddings).head(5)

Wall time: 167 ms


,id,simscore,result
0,0,1.000000,la carte users reported intermittently getting...
9,9,1.000000,planning book unavailable business users
28,28,1.000000,sbc users getting offers intermittently
5,5,0.692157,users able post goods movement
13,13,0.692157,users able access lilo ui
